![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll save SQL files as multiline string variables that they can then use to create the database on their end. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import psycopg2
from config import username, password

# Read in bank_marketing.csv
df = pd.read_csv('data/bank_marketing.csv')

# Display df
df.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [2]:
# Create the client dataframe
client = df[['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'housing', 'loan']]

# Rename the column 'client_id' to 'id'
client = client.rename(columns={'client_id': 'id'})

# Display results
client.head()

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic.4y,no,no,no
1,1,57,services,married,high.school,unknown,no,no
2,2,37,services,married,high.school,no,yes,no
3,3,40,admin.,married,basic.6y,no,no,no
4,4,56,services,married,high.school,no,no,yes


In [3]:
# Create the campaign dataframe
campaign = df[['client_id', 'campaign', 'month', 'day', 'duration', 'pdays', 'previous', 'poutcome', 'y']]

# Rename columns
campaign = campaign.rename(columns={'duration': 'contact_duration', 'previous': 'previous_campaign_contacts', 'y': 'campaign_outcome', 'campaign': 'number_contacts', 'poutcome': 'previous_outcome'})

# Display results
campaign.head()

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,nonexistent,no
1,1,1,may,19,149,999,0,nonexistent,no
2,2,1,may,23,226,999,0,nonexistent,no
3,3,1,may,27,151,999,0,nonexistent,no
4,4,1,may,3,307,999,0,nonexistent,no


In [4]:
# Create the economics dataframe
economics = df[['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']]

# Rename columns
economics = economics.rename(columns={'euribor3m': 'euribor_three_months', 'nr_employed': 'number_employed'})

# Display results
economics

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0
...,...,...,...,...,...
41183,41183,-1.1,94.767,1.028,4963.6
41184,41184,-1.1,94.767,1.028,4963.6
41185,41185,-1.1,94.767,1.028,4963.6
41186,41186,-1.1,94.767,1.028,4963.6


In [5]:
# Fuction to change unknown to Numpy's null values
def replace_unknown_with_nan(df, columns):
    """Replace 'unknown' values in the specified dataframe columns with numpy NaN."""
    for column in columns:    
        df[column] = df[column].replace('unknown', np.NaN)
    return df

In [6]:
# Clean the 'education' column in client df 
client['education'] = client['education'].str.replace('.', '_')

# Create a list of columns to clean
columns_to_clean = ['credit_default', 'education', 'housing', 'loan']

# Apply function to columns in client
client = replace_unknown_with_nan(client, columns_to_clean)

# Preview df
client.head()

/var/folders/44/0wp3m6953qq8ht7m3krhf89c0000gn/T/ipykernel_39000/1435348800.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  client['education'] = client['education'].str.replace('.', '_')


,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,NaN,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin.,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes


In [7]:
# Create a dictionary
map_dict = {'yes': 1, 'no': 0}
map_dict2 = {'success': 1, 'failure': 0}

# Change the 'campaign_outcome' column to binary values
campaign['campaign_outcome'] = campaign['campaign_outcome'].map(map_dict)

# Change the 'previous_outcome' column to binary values
campaign['previous_outcome'] = campaign['previous_outcome'].map(map_dict2)

# Change 'nonexistent' to Numpy's null values in 'previous_outcome' column
campaign['previous_outcome'] = campaign['previous_outcome'].replace('nonexistent', np.NaN)

# Change '0.0' to 0 in 'previous_outcome' column
campaign['previous_outcome'] = campaign['previous_outcome'].replace('0.0', 0)

# Change 'failure' to Numpy's null values in 'previous_campaign_contacts' column
campaign['previous_campaign_contacts'] = campaign['previous_campaign_contacts'].replace('failure', np.NaN)

# Add a column called campaign_id in campaign
campaign['campaign_id'] = '1'

# Capitalize the values of 'month'
campaign['month'] = campaign['month'].str.capitalize()

# Add a column called 'year'
campaign['year'] = '2022'

# Convert the 'day' column to string
campaign['day'] = campaign['day'].astype(str)

# Create a column called 'last_contact_date'
campaign['last_contact_date'] = campaign['year'] + '-' + campaign['month'] + '-' + campaign['day']

# Convert 'last_contact_date' to datetime
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], format='%Y-%b-%d')

# Delete unnecessary columns
campaign.drop(columns=['month', 'day', 'year'], inplace=True)

# Display results
campaign.head()

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,1,261,999,0,NaN,0,1,2022-05-13
1,1,1,149,999,0,NaN,0,1,2022-05-19
2,2,1,226,999,0,NaN,0,1,2022-05-23
3,3,1,151,999,0,NaN,0,1,2022-05-27
4,4,1,307,999,0,NaN,0,1,2022-05-03


In [8]:
# Save the three dataframes to csv files
client.to_csv('data/client.csv',index=False)
campaign.to_csv('data/campaign.csv',index=False)
economics.to_csv('data/economics.csv',index=False)

In [9]:
conn = psycopg2.connect(
    database="bank",
    user=username,
    password=password,
    host="localhost",
    port="5432"
)
cur = conn.cursor()  # This will allow you to execute SQL commands.


In [10]:
# SQL command to drop a table if it exists
drop_table_command = "DROP TABLE IF EXISTS client CASCADE;"

# Execute the DROP TABLE command
cur.execute(drop_table_command)

# Create the client table in sql 
client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
"""

# Execute the CREATE TABLE command
cur.execute(client_table)

# Commit the transaction
conn.commit()

In [11]:
copy_data_command = """
COPY client FROM STDIN WITH CSV HEADER DELIMITER ',';
"""
with open('data/client.csv', 'r') as file:
    cur.copy_expert(copy_data_command, file)
    conn.commit()


In [14]:
conn.rollback() 

In [15]:
# SQL command to drop a table if it exists
drop_table_command = "DROP TABLE IF EXISTS campaign CASCADE;"

# Execute the DROP TABLE command
cur.execute(drop_table_command)

# Create the campaign table in sql
campaign_table = """
CREATE TABLE campaign ( 
    client_id SERIAL references client (id),
    number_contacts INTEGER,
    contact_duration INTEGER,
    pdays INTEGER,
    previous_campaign_contacts INTEGER,
    previous_outcome FLOAT,
    campaign_outcome BOOLEAN,
    campaign_id INTEGER,
    last_contact_date DATE
);
"""

# Execute the CREATE TABLE command
cur.execute(campaign_table)

# Commit the transaction
conn.commit()

In [16]:
copy_data_command = """
COPY campaign FROM STDIN WITH CSV HEADER DELIMITER ',';
"""
with open('data/campaign.csv', 'r') as file:
    cur.copy_expert(copy_data_command, file)
    conn.commit()

In [17]:
# SQL command to drop a table if it exists
drop_table_command = "DROP TABLE IF EXISTS economics CASCADE;"

# Execute the DROP TABLE command
cur.execute(drop_table_command)

# Create the economics table in sql
economics = """
CREATE TABLE economics (
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
); 
"""
# Execute the CREATE TABLE command
cur.execute(economics)

# Commit the transaction
conn.commit()

In [18]:
copy_data_command = """
COPY economics FROM STDIN WITH CSV HEADER DELIMITER ',';
"""
with open('data/campaign.csv', 'r') as file:
    cur.copy_expert(copy_data_command, file)
    conn.commit()

BadCopyFileFormat: extra data after last expected column
CONTEXT:  COPY economics, line 2: "0,1,261,999,0,,0,1,2022-05-13"
